In [1]:
from google.colab import drive

- cvするのと全データ学習するのとどちらがいいのか調べる

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls /content/drive/MyDrive/4sq/input/

lgb_012.lgb	       train.csv	   train_data_1_1.zip  train_data3.csv
lgb_01.lgb	       train_data_0_1.csv  train_data_1_2.csv  train_data4.csv
lgb_02.lgb	       train_data_0_2.csv  train_data_1_3.csv  train_data5.csv
lgb_12.lgb	       train_data_0_3.csv  train_data_1_4.csv  tv_ids_d.npy
pairs.csv	       train_data_0_4.csv  train_data_1_5.csv
sample_submission.csv  train_data_0_5.csv  train_data1.csv
test.csv	       train_data_1_1.csv  train_data2.csv


In [4]:
!ls /content/drive/MyDrive/4sq/output/

catboost_0.pkl	lgb_0.lgb  lgb_1.lgb  lgb_2.lgb  lgb_4.lgb
catboost_1.pkl	lgb_0.pkl  lgb_1.pkl  lgb_3.lgb  lgb_tmp.lgb


In [5]:
!python -m pip install "dask[complete]"
! pip3 install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
## Imports
import warnings
warnings.filterwarnings('ignore')

import os
import gc
import time
import random
import difflib
import multiprocessing
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

import dask.dataframe as dd

from catboost import Pool
from catboost import CatBoostClassifier, Pool, MetricVisualizer
from sklearn import metrics

In [7]:
input_type = {
'id':                    'object',
'match_id':              'object',
'kdist':                 'float32',
'kneighbors':            'float32',
'kdist_country':         'float32',
'latdiff':                 'float32',
'londiff':                 'float32',
'manhattan':            'float32',
'euclidean':              'float32',
'haversine':              'float32',
'kneighbors_country':    'float32',
'label':                 'bool',
'name_sim':              'float32',
'name_gesh':             'float32',
'name_leven':            'float32',
'name_jaro':             'float32',
'name_lcs':              'float32',
'name_len_diff':         'uint8',
'name_nleven':           'float32',
'name_nlcsk':            'float32',
'name_nlcs':             'float32',
'address_sim':           'float32',
'address_gesh':          'float32',
'address_leven':         'float32',
'address_jaro':          'float32',
'address_lcs':           'float32',
'address_len_diff':      'uint8',
'address_nleven':        'float32',
'address_nlcsk':         'float32',
'address_nlcs':          'float32',
'city_gesh':             'float32',
'city_leven':            'float32',
'city_jaro':             'float32',
'city_lcs':              'float32',
'city_len_diff':         'uint8',
'city_nleven':           'float32',
'city_nlcsk':            'float32',
'city_nlcs':             'float32',
'state_sim':             'float32',
'state_gesh':            'float32',
'state_leven':           'float32',
'state_jaro':            'float32',
'state_lcs':             'float32',
'state_len_diff':        'uint8',
'state_nleven':          'float32',
'state_nlcsk':           'float32',
'state_nlcs':            'float32',
'zip_gesh':              'float32',
'zip_leven':             'float32',
'zip_jaro':              'float32',
'zip_lcs':               'float32',
'url_sim':               'float32',
'url_gesh':              'float32',
'url_leven':             'float32',
'url_jaro':              'float32',
'url_lcs':               'float32',
'url_len_diff':          'uint8',
'url_nleven':            'float32',
'url_nlcsk':             'float32',
'url_nlcs':              'float32',
'phone_gesh':            'float32',
'phone_leven':           'float32',
'phone_jaro':            'float32',
'phone_lcs':             'float32',
'categories_sim':        'float32',
'categories_gesh':       'float32',
'categories_leven':      'float32',
'categories_jaro':       'float32',
'categories_lcs':        'float32',
'categories_len_diff':   'uint8',
'categories_nleven':     'float32',
'categories_nlcsk':      'float32',
'categories_nlcs':       'float32',
'country_sim':           'float32',
'country_gesh':          'float32',
'country_leven':         'float32',
'country_jaro':          'float32',
'country_lcs':           'float32',
'country_len_diff':      'bool',
'country_nleven':        'float32',
'country_nlcsk':         'float32',
'country_nlcs':          'float32',
'kdist_diff':            'float32',
'kneighbors_mean':       'float32',
'country_x':             'object',
'country_y':             'object',
}
cat_list = ['country_x', 'country_y']

In [129]:
CV_NUM = 2
THRESHOLD = 0.50
NUM_SPLIT = 10

In [130]:
import pickle
lgb_model_list = []
for i in range(CV_NUM):
    #lgb_model_path = f'drive/MyDrive/4sq/output/lgb_{i}.lgb'
    #lgb_model = lgb.Booster(model_file = lgb_model_path)
    lgb_model_path = f'drive/MyDrive/4sq/output/lgb_{i}.pkl'
    lgb_model = pickle.load(open(lgb_model_path, 'rb'))
    lgb_model_list.append(lgb_model)

In [131]:
catboost_model_list = []
for i in range(CV_NUM):
    catboost_model_path = f'drive/MyDrive/4sq/output/catboost_{i}.pkl'
    catboost_model = pickle.load(open(catboost_model_path, 'rb'))
    catboost_model_list.append(catboost_model)

In [132]:
#valid_list = [dd.read_csv(f'drive/MyDrive/4sq/input/train_data_1_{i}.csv', dtype=input_type).compute() for i in range(1,6)]
#valid_list = [dd.read_csv(f'drive/MyDrive/4sq/input/train_data_1_{i}.csv', dtype=input_type).compute() for i in range(1,3)]
#valid = pd.concat([valid_list[0], valid_list[1],valid_list[2], valid_list[3], valid_list[4]], ignore_index=True)
valid = dd.read_csv('drive/MyDrive/4sq/input/train_data_1_1.csv', dtype=input_type).compute()
#valid = dd.read_csv('drive/MyDrive/4sq/input/train_data_0_1.csv', dtype=input_type).compute()

In [133]:
unique_id = valid['id'].unique().tolist()

In [134]:
len(unique_id)

113881

In [135]:
# importanceの低い素性を削る
valid = valid.drop(['country_jaro', 'country_nlcsk', 'country_lcs', 'country_len_diff', 'country_nlcs', 'country_nleven', 'country_leven'], axis=1)
# カテゴリに変換
valid['country_x'] = valid['country_x'].fillna('nanstr')
valid['country_y'] = valid['country_y'].fillna('nanstr')
valid['country_x'] = valid['country_x'].astype('category')
valid['country_y'] = valid['country_y'].astype('category')
valid = valid.query('id != match_id')

In [136]:
valid

,id,match_id,kdist,kneighbors,kdist_country,kneighbors_country,label,latdiff,londiff,manhattan,...,categories_jaro,categories_lcs,categories_len_diff,categories_nleven,categories_nlcsk,categories_nlcs,country_sim,country_gesh,kdist_diff,kneighbors_mean
21,E_000260dd24cecf,E_919a37430bea7a,0.0,0.0,0.0,1.0,True,0.000000,0.000000,0.000000,...,1.000000,15.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,0.5
140,E_000fa08832fbd7,E_0146e712d66bb8,0.0,0.0,0.0,15.0,False,0.000000,0.000000,0.000000,...,1.000000,21.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,7.5
282,E_00219328bc7ecb,E_6f272b9533b508,0.0,0.0,0.0,0.0,True,0.000000,0.000000,0.000000,...,1.000000,12.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,0.0
308,E_0024fc2bd72ad5,E_b2cc39e9379c00,0.0,0.0,0.0,1.0,True,0.000000,0.000000,0.000000,...,NaN,NaN,0,NaN,NaN,NaN,1.0,1.0,NaN,0.5
682,E_005143e7c63ed9,E_bc96ffb9eccdb4,0.0,0.0,0.0,0.0,False,0.000000,0.000000,0.000000,...,0.538047,7.0,12,0.766667,0.233333,0.388889,1.0,1.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82360,E_330ffca9bd67a0,E_8417f3957e9a10,NaN,NaN,NaN,NaN,False,0.170272,0.088559,0.258831,...,1.000000,12.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN
82361,E_330ffca9bd67a0,E_148c43141b551d,NaN,NaN,NaN,NaN,False,0.034695,0.036551,0.071247,...,1.000000,12.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN
82362,E_330ffca9bd67a0,E_a9d775cf98c10a,NaN,NaN,NaN,NaN,False,0.016849,0.031669,0.048518,...,1.000000,12.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN
82363,E_330ffca9bd67a0,E_787bea00dd6478,NaN,NaN,NaN,NaN,False,0.212640,0.029969,0.242609,...,1.000000,12.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN


In [137]:
#valid_x = valid.drop(['label'], axis=1)
#valid_y = valid['label']
#valid_x = valid_x.drop(['id', 'match_id'], axis=1)

In [138]:
# inputのsizeによって微妙に変わっちゃう
def reduce_mem_usage(df, verbose=True):
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2
  
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)
          else:
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)
  
  end_mem = df.memory_usage().sum() / 1024**2
  print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
  print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
  
  return df

#valid_x = reduce_mem_usage(valid_x)

In [139]:
lgb_model_list[0].best_iteration

1074

In [140]:
catboost_model_list[0].get_params()

{'bagging_temperature': 0.10721404311739535,
 'depth': 10,
 'eval_metric': 'AUC',
 'iterations': 1000,
 'learning_rate': 0.1,
 'loss_function': 'Logloss',
 'od_type': 'IncToDec',
 'od_wait': 28,
 'random_seed': 0,
 'random_strength': 1,
 'task_type': 'GPU'}

In [141]:
catboost_model_list[0]

## CV

In [142]:
# calcuate cv
#pred = lgb_model_list[0].predict(valid_x, num_iteration=lgb_model_list[0].best_iteration)
#for i in range(1, CV_NUM):
#    pred_tmp = lgb_model_list[i].predict(valid_x, num_iteration=lgb_model_list[i].best_iteration)
#    pred = np.add(pred, pred_tmp)          
#pred = np.array([i / CV_NUM for i in pred])  

In [143]:
#valid['pred'] = pred
#pred_df = valid[valid['pred'] > THRESHOLD][['id', 'match_id']]

In [144]:
#pred.mean()

In [145]:
len(unique_id) 

113881

In [160]:
count = 0
start_row = 0
pred_df = pd.DataFrame()
num_split_id = len(unique_id) // NUM_SPLIT
#pred_arr_lgb = np.array([])
#pred_arr_cat = np.array([])

for k in range(1, NUM_SPLIT + 1):
    print('Current split: %s' % k)
    end_row = start_row + num_split_id
    if k < NUM_SPLIT:
        cur_id = unique_id[start_row : end_row]
        cur_data = valid[valid['id'].isin(cur_id)]
    else:
        cur_id = unique_id[start_row: ]
        cur_data = valid[valid['id'].isin(cur_id)]

    #cur_data = reduce_mem_usage(cur_data)  
    cur_data_tmp = cur_data.drop(['id', 'match_id'], axis=1)
    cur_data_tmp = cur_data_tmp.drop(['label'], axis=1)

    lgb_pred = lgb_model_list[0].predict(cur_data_tmp, num_iteration=lgb_model_list[0].best_iteration)
    # LGBM
    for i in range(1, CV_NUM):
        pred_tmp = lgb_model_list[i].predict(cur_data_tmp, num_iteration=lgb_model_list[i].best_iteration)
        lgb_pred = np.add(lgb_pred, pred_tmp) 
    # catboost
    categorical_features_indices = np.where(cur_data_tmp.dtypes == 'category')[0] 
    pool = Pool(cur_data_tmp, cat_features=categorical_features_indices)   
    cat_pred = catboost_model_list[0].predict_proba(pool)[:,1]
    for i in range(1, CV_NUM):
        pred_tmp = catboost_model_list[i].predict_proba(pool)[:,1]
        cat_pred = np.add(cat_pred, pred_tmp)

    arr_lgb = np.array([i / CV_NUM for i in lgb_pred])
    arr_cat = np.array([i / CV_NUM for i in cat_pred])
    W = 0.5
    arr = np.array([(i * W) + j * (1 - W) for i,j in zip(arr_lgb, arr_cat)])

    print('lgb pred mean:', arr_lgb.mean())
    print('cat pred mean:', arr_cat.mean())
    print('avg pred mean:', arr.mean())

    #print('pred_arr_lgb:', len(pred_arr_lgb))
    #if len(pred_arr_lgb) == 0:
    #    pred_arr_lgb = arr_lgb
    #    pred_arr_cat = arr_cat
    #else:
    #    pred_arr_lgb = np.hstack((pred_arr_lgb, arr_lgb))
    #    pred_arr_cat = np.hstack((pred_arr_cat, arr_cat))        
    cur_data['pred'] = arr
    cur_pred_df = cur_data[cur_data['pred'] > THRESHOLD][['id', 'match_id']]
    pred_df = pd.concat([pred_df, cur_pred_df])
    start_row = end_row
    count += len(cur_data)

    del cur_data, cur_pred_df
    gc.collect()

print(count)


Current split: 1
lgb pred mean: 0.025811768566320233
cat pred mean: 0.026158872525829706
avg pred mean: 0.025985320546074983
Current split: 2
lgb pred mean: 0.026531036539591765
cat pred mean: 0.027002521393519684
avg pred mean: 0.026766778966555733
Current split: 3
lgb pred mean: 0.026364397867375444
cat pred mean: 0.02671831351331692
avg pred mean: 0.026541355690346186
Current split: 4
lgb pred mean: 0.025467205248157744
cat pred mean: 0.02572900726867342
avg pred mean: 0.025598106258415586
Current split: 5
lgb pred mean: 0.025403611124875092
cat pred mean: 0.02581143327656895
avg pred mean: 0.025607522200722033
Current split: 6
lgb pred mean: 0.02525721100210389
cat pred mean: 0.025634578197609226
avg pred mean: 0.02544589459985655
Current split: 7
lgb pred mean: 0.026122236663231595
cat pred mean: 0.02651876194016857
avg pred mean: 0.026320499301700082
Current split: 8
lgb pred mean: 0.026957785968083514
cat pred mean: 0.027475918609323014
avg pred mean: 0.02721685228870325
Current

In [147]:
#valid['pred'] = pred_arr_lgb

In [148]:
#valid_tmp = valid
#valid_tmp = valid_tmp[['id', 'match_id', 'label', 'pred']]

In [149]:
# IoUを計算する時は自分自身も加えるのでuniqueなidを計算しておく
#unum = len(valid_tmp['id'].unique())

In [150]:
valid

,id,match_id,kdist,kneighbors,kdist_country,kneighbors_country,label,latdiff,londiff,manhattan,...,categories_jaro,categories_lcs,categories_len_diff,categories_nleven,categories_nlcsk,categories_nlcs,country_sim,country_gesh,kdist_diff,kneighbors_mean
21,E_000260dd24cecf,E_919a37430bea7a,0.0,0.0,0.0,1.0,True,0.000000,0.000000,0.000000,...,1.000000,15.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,0.5
140,E_000fa08832fbd7,E_0146e712d66bb8,0.0,0.0,0.0,15.0,False,0.000000,0.000000,0.000000,...,1.000000,21.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,7.5
282,E_00219328bc7ecb,E_6f272b9533b508,0.0,0.0,0.0,0.0,True,0.000000,0.000000,0.000000,...,1.000000,12.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,0.0
308,E_0024fc2bd72ad5,E_b2cc39e9379c00,0.0,0.0,0.0,1.0,True,0.000000,0.000000,0.000000,...,NaN,NaN,0,NaN,NaN,NaN,1.0,1.0,NaN,0.5
682,E_005143e7c63ed9,E_bc96ffb9eccdb4,0.0,0.0,0.0,0.0,False,0.000000,0.000000,0.000000,...,0.538047,7.0,12,0.766667,0.233333,0.388889,1.0,1.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82360,E_330ffca9bd67a0,E_8417f3957e9a10,NaN,NaN,NaN,NaN,False,0.170272,0.088559,0.258831,...,1.000000,12.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN
82361,E_330ffca9bd67a0,E_148c43141b551d,NaN,NaN,NaN,NaN,False,0.034695,0.036551,0.071247,...,1.000000,12.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN
82362,E_330ffca9bd67a0,E_a9d775cf98c10a,NaN,NaN,NaN,NaN,False,0.016849,0.031669,0.048518,...,1.000000,12.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN
82363,E_330ffca9bd67a0,E_787bea00dd6478,NaN,NaN,NaN,NaN,False,0.212640,0.029969,0.242609,...,1.000000,12.0,0,0.000000,1.000000,1.000000,1.0,1.0,NaN,NaN


In [151]:
#pred_arr_cat

In [152]:
"""
for THRESHOLD in (0.4, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70):
  valid_tmp = valid
  valid_tmp = valid_tmp[['id', 'match_id', 'label']]
  valid_tmp['pred'] = pred_arr_lgb
  unum = len(valid_tmp['id'].unique())
  print("THRESHOLD=", THRESHOLD, "****************")
  true_num = len(valid_tmp.query('label==1'))
  pred_num = len(valid_tmp.query(f'pred>{THRESHOLD}'))
  a = len(valid_tmp.query(f'label==1 or pred>{THRESHOLD}'))
  b = len(valid_tmp.query(f'label==1 and pred>{THRESHOLD}'))
  print("true_num:", true_num, "pred_num:", pred_num)
  print('a, b=', a, b)
  print('IoU=', (b + unum) / (a + unum))
##print('IoU=', b/a)
"""

'\nfor THRESHOLD in (0.4, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70):\n  valid_tmp = valid\n  valid_tmp = valid_tmp[[\'id\', \'match_id\', \'label\']]\n  valid_tmp[\'pred\'] = pred_arr_lgb\n  unum = len(valid_tmp[\'id\'].unique())\n  print("THRESHOLD=", THRESHOLD, "****************")\n  true_num = len(valid_tmp.query(\'label==1\'))\n  pred_num = len(valid_tmp.query(f\'pred>{THRESHOLD}\'))\n  a = len(valid_tmp.query(f\'label==1 or pred>{THRESHOLD}\'))\n  b = len(valid_tmp.query(f\'label==1 and pred>{THRESHOLD}\'))\n  print("true_num:", true_num, "pred_num:", pred_num)\n  print(\'a, b=\', a, b)\n  print(\'IoU=\', (b + unum) / (a + unum))\n##print(\'IoU=\', b/a)\n'

### 元データ読み取り
- IoU計算用

In [153]:
## Data load
data_root = '/content/drive/MyDrive/4sq/input/'
data = pd.read_csv(os.path.join(data_root, 'train.csv'))

In [154]:
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

In [155]:
id2poi = get_id2poi(data) # idからPoEを取得
poi2ids = get_poi2ids(data) # PoEからidのsetを取得

## Calculate IoU

In [156]:
def post_process(df):
    id2match = dict(zip(df['id'].values, df['matches'].str.split()))

    for base, match in df[['id', 'matches']].values:
        match = match.split()
        if len(match) == 1:        
            continue
        # matchをseedにした時にbaseがなければ加える（対称性を担保）
        # 今回はデータを絞っててmatchをseedにしてるものがあるとは限らないので条件追加
        for m in match:
            #if base not in id2match[m]:
            if m in id2match and base not in id2match[m]:
                id2match[m].append(base)
    df['matches'] = df['id'].map(id2match).map(' '.join)
    return df 

In [157]:
def post_process2(df):
    """ if A matches B and B matches C, then A matchs C and C matches A """
    match_map = df.set_index('id')['matches'].to_dict()
    matches = []
    ids_check = []
    for id, id_df in tqdm(df.groupby('id')):
        match = id_df['matches'].str.split().iloc[0]
        new_match = []
        for match_id in match:
            if match_id != id:
                new_match += match_map[match_id].split()

        if len(new_match) > 0:
            match += new_match
            match = list(dict.fromkeys(match)) # remove duplicates
        match = ' '.join(match)
        matches.append(match)
        ids_check.append(id)

    df.loc[df['id'].isin(ids_check), 'matches'] = matches
    return df

In [164]:
out_df = pd.DataFrame()
out_df['id'] = valid['id'].unique().tolist()
out_df['match_id'] = out_df['id']

In [165]:
out_df = pd.concat([out_df, pred_df])
out_df = out_df.groupby('id')['match_id'].apply(list).reset_index()
out_df['matches'] = out_df['match_id'].apply(lambda x: ' '.join(set(x)))
out_df = post_process(out_df)
#out_df = post_process2(out_df)

In [166]:
def get_score(input_df: pd.DataFrame):
    scores = []
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

In [167]:
iou_score = get_score(out_df)
print('IoU score: %s' % iou_score)

IoU score: 0.8678238650984479


In [40]:
# kernel 0.8502410162084575

# valid1つだけ
# cat 0.45, post on 0.8482415733162958 
# cat 0.45, post off 0.847826422914576 
# cat 0.50, post on 0.8480137343805896
# cat 0.50, post off 0.8474988972191119

# lgb 0.45, post on 0.8485343632201866 
# lgb 0.45, post off 0.8481181127249435

# lgb*0.4+cat*0.6 0.45 post on 0.8501585991102277
# lgb*0.5+cat*0.5 0.45 post on 0.8503278293079118


# valid all
# lgb*0.5+cat*0.5 0.45 post on 0.8521011366330579
# lgb*0.5+cat*0.5 0.45 post off 0.8497539646314036
# lgb*0.5+cat*0.5 0.45 post2 on 0.8500853017375279
# lgb*0.5+cat*0.5 0.50 post2 on 0.8521180841749597
